In [1]:
# add all necessary imports
from tqdm import tqdm
from convokit import Corpus, Speaker, Utterance
from collections import defaultdict
import pandas as pd

Need to convert alldyads.xlsx into dataframe with columns compatible with corpus structure to makr object
Need:
# Convert Dataframe file

## methods:

#conversationID based on 1st utterance of df

### def toUtterance():
get all primary info from formatted dataframe column

- id
- speaker
- conversation_id
- reply_to
- timestamp
- text



#### Metadata:
{metadata} (per speaker)
metadata: country code column
### def toSpeakers():
- need to convert each row to have unique buyer/seller ID
- check if there are duplicate participants allowed
- isAI (if seller is AI)
#### Metadata:
- country code (b_country, s_country)
- self reports (later)


### def toConversations():
convo-id : {metadata}
#### Metadata:
- outcome of dipsute

  
### def toCorpus():
- save for later
  
### def toIndex():
- save for later

# CorpusConverter
- formattedChat

In [2]:
from modules.DataPreprocesser import DataPreprocesser

In [3]:
filepath = "/Users/mishkin/Desktop/Research/Convo_Kit/ConvoKit_Disputes/data/alldyads.csv"
data_preprocessor = DataPreprocesser(filepath)
data_preprocessor.addParsedTextColumn("formattedChat", "parsed_dialog")

data = data_preprocessor.df.copy() 
data["parsed_dialog"]
orig_data =pd.read_csv(filepath)
print(data.shape)
print(orig_data.shape)

2882
(2882, 51)
(2891, 50)


### Example of parsedText Column

In [4]:
data_preprocessor.getDataframe()["parsed_dialog"][0]

[{'timestamp': 'nan',
  'speaker': 'Seller',
  'message': 'Your sudden demand for a refund is unwarranted. Our product description is crystal clear, and we stand by our policy. Your behavior is disappointing, and your negative review is unfounded.',
  'value': None,
  'uttidx': 0,
  'speaker_id': 'Seller_0',
  'row_idx': 0,
  'match_idx': False,
  'Case Match Type': None},
 {'timestamp': 1699388150,
  'speaker': 'Buyer',
  'message': 'This was not the item I ordered, please provide a full refund.  I will ship the item back.',
  'value': None,
  'uttidx': 1,
  'speaker_id': 'Buyer_0',
  'row_idx': 0,
  'match_idx': False,
  'Case Match Type': None},
 {'timestamp': 1699388170,
  'speaker': 'Seller',
  'message': 'How about a compromise? I can offer you a partial refund, and in exchange, you would remove your negative review that wrongly portrays my business.',
  'value': None,
  'uttidx': 2,
  'speaker_id': 'Seller_0',
  'row_idx': 0,
  'match_idx': False,
  'Case Match Type': None},
 {'

### Example of Utterance Dataframe

In [5]:
data_preprocessor.getUtterancesDF()

,timestamp,speaker,message,value,uttidx,speaker_id,row_idx,match_idx,Case Match Type,convo_len
0,nan,Seller,Your sudden demand for a refund is unwarranted...,None,0,Seller_0,0,False,None,16
1,1699388150,Buyer,"This was not the item I ordered, please provid...",None,1,Buyer_0,0,False,None,16
2,1699388170,Seller,How about a compromise? I can offer you a part...,None,2,Seller_0,0,False,None,16
3,1699388217,Buyer,"No, this was not the item I ordered. I have a ...",None,3,Buyer_0,0,False,None,16
4,1699388235,Seller,"I can scrutinize your claim, but this requires...",None,4,Seller_0,0,False,None,16
...,...,...,...,...,...,...,...,...,...,...
39841,1718879890,Buyer,Thank you I appreciate that. I shall remove m...,None,5,Buyer_2881,2881,False,None,10
39842,1718879909,Seller,"Deal. Once I see changes on your end, I'll ini...",None,6,Seller_2881,2881,False,None,10
39843,1718879998,Buyer,I understand - please do follow through. I ha...,None,7,Buyer_2881,2881,False,None,10
39844,1718880018,Seller,"You have my word, I will follow through. A sna...",None,8,Seller_2881,2881,False,None,10


# Dataset Analysis

### Empty formattedChats

In [6]:
#get empty formatted chats
empty_rows = data[data["formattedChat"].isna() | (data["formattedChat"] == "")]
print(empty_rows.index)

Index([], dtype='int64')


### Duplicated Formatted Chats
duplicate chats only associated with empty formattedChats

In [7]:
duplicates = data[data['formattedChat'].duplicated()]
print(duplicates.index)
duplicates

Index([], dtype='int64')


,b_country,b_RankBuy_1,b_RankBuy_2,b_RankBuy_3,b_RankBuy_4,b_Tact_1,b_Tact_2,b_Tact_3,b_Tact_4,b_Tact_5,...,s_Ai4me-why,s_Use emotions,s_AI4u,s_PlayAI.1,s_AI-cues.1,is_AI,formattedChat,b_insight,s_insight,parsed_dialog


## Find all Formatted Chats with Dispute Outcomes
can select "Walk Away" after at least 8 chat messages
Buyers always begin the negotiation (unless one participant is an AI, then the AI always goes first). Participants alternate sending messages; the participant’s interface becomes inactive while awaiting a new message.

**Info:**
- Many chats have "Your sudden deman is unwarranted" for seller
- Many chats have "Your response is utterly unacceptable" for buyer

**Not very clear when a formatted chat is considered to have a dispute outcome. Possible Cases:**
- submitted agreement, but submitted modified terms -> other party rejects
- submitted agreement, but no clicked response from other party (Walk Away or Accept Deal)
- submitted agreement, but manual response from other party that needs extra interpretation (e.g. "I don't like this")
- submitted agreement, but flawed input logic (see below)
- submitted agreement missing


**Other formattedChat Concerns:**
- Gemini texts start with seller? (Buyer should always lead)
- Other]: used as initial text for Seller respones in Gemini chats
- some repeated text in Gemini chats in parsed_dialogs

**Other Notes Concerns:**
    - Time Stamp Idle Time

### For our analysis Criteria:
- Remove all chats missing "Walk Away" or "Accept Deal" from the formattedChat. Case Sensistivy applies, as it is automatic reply by Lioness Labs software prompted by user selection.

### Differences in "I Walk Away" Case Sensitivity

In [13]:

#I Walk Away versua Walk Away
filtered_rows_df_i = data_preprocessor.getUtterancesDF()[data_preprocessor.getUtterancesDF()["message"].str.contains(r"\bI Walk Away\b", case=False, na=False, regex=True)]
filtered_rows_df_I = data_preprocessor.getUtterancesDF()[data_preprocessor.getUtterancesDF()["message"].str.contains(r"\bI Walk Away\b", case=True, na=False, regex=True)]
filtered_rows_df_w = data_preprocessor.getUtterancesDF()[data_preprocessor.getUtterancesDF()["message"].str.contains(r"\bWalk Away\b", case=False, na=False, regex=True)]
filtered_rows_df_W = data_preprocessor.getUtterancesDF()[data_preprocessor.getUtterancesDF()["message"].str.contains(r"\bWalk Away\b", case=True, na=False, regex=True)]

print(len(filtered_rows_df_i))
print(len(filtered_rows_df_I))
print(len(filtered_rows_df_w))
print(len(filtered_rows_df_W))


data_preprocessor.filterMatches('message', "I Walk Away")
# data_preprocessor.filterMatches('message', "Walk Away")
data_preprocessor.text_matches_new["I Walk Away"][1]
# #add the convolen only to the text_matches_new column
# data_preprocessor.getMatchedUtterancesDF("I Walk Away")
# data_preprocessor.getMatchedUtterancesDF("Walk Away")


data_preprocessor.groupbyMatchConvoStat("I Walk Away", "row_idx", "convo_len", ['sum'])
# data_preprocessor.getMatchedConvoDF("I Walk Away")
# data_preprocessor.text_matches_new["I Walk Away"][1]
# data_preprocessor.getConvoMatchesByCase("I Walk Away").sum()
# data_preprocessor.getTTR("I Walk Away")

#data_preprocessor.text_matches_new["I Walk Away"][0]
#data_preprocessor.getUtterancesDF()
#data_preprocessor.getMatchesByCase("I Walk Away")




473
469
553
470


/Users/mishkin/Desktop/Research/Convo_Kit/ConvoKit_Disputes/dataprocessing/modules/DataPreprocesser.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_convo[stat_col] = self.text_matches_new[value_key][1][stat_col]  # Assuming this exists


,sum
row_idx,
0,256
1,196
2,144
3,144
4,196
...,...
2877,256
2878,100
2879,225


In [9]:
data_preprocessor.getDataframe()

,b_country,b_RankBuy_1,b_RankBuy_2,b_RankBuy_3,b_RankBuy_4,b_Tact_1,b_Tact_2,b_Tact_3,b_Tact_4,b_Tact_5,...,s_Ai4me-why,s_Use emotions,s_AI4u,s_PlayAI.1,s_AI-cues.1,is_AI,formattedChat,b_insight,s_insight,parsed_dialog
0,187.0,80.0,12.0,5.0,3.0,1.0,4.0,5.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,True,nan Seller: Your sudden demand for a refund is...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Seller', 'me..."
1,187.0,40.0,10.0,31.0,19.0,3.0,1.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,False,1699388451 Buyer: Hi there! I believe I receiv...,3.0,1.0,"[{'timestamp': 1699388451, 'speaker': 'Buyer',..."
2,187.0,40.0,10.0,30.0,20.0,2.0,4.0,4.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,True,nan Seller: Your sudden demand for a refund is...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Seller', 'me..."
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,nan Buyer: Your response is utterly unacceptab...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Buyer', 'mes..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,nan Buyer: Your response is utterly unacceptab...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Buyer', 'mes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,161.0,50.0,10.0,20.0,20.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,nan Seller: Your sudden demand for a refund is...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Seller', 'me..."
2878,185.0,60.0,10.0,30.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,nan Seller: Your sudden demand for a refund is...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Seller', 'me..."
2879,185.0,50.0,10.0,30.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,nan Seller: Your sudden demand for a refund is...,NaN,NaN,"[{'timestamp': 'nan', 'speaker': 'Seller', 'me..."
2880,185.0,60.0,10.0,25.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,It souunded too good to be true yet so human a...,False,"1718878682 Buyer: Hi there, thank you for send...",3.0,4.0,"[{'timestamp': 1718878682, 'speaker': 'Buyer',..."


### "I Walk Away" versus "Walk Away" Appearances in Dispute (Positonal Correlation) (todo)

In [10]:
#I Walk Away versua Walk Away
import pprint as pp

only_walk = data_preprocessor.filterRows("formattedChat","Walk Away", "I Walk Away", False, False)

matches_Walk= data_preprocessor.filterMatches("message", "I Walk Away")
matches_IWalk = data_preprocessor.filterMatches("message", "Walk Away")

data_preprocessor.getMatchedUtterancesDF("I Walk Away")
data_preprocessor.getMatchedUtterancesDF("Walk Away")

mean_1 = data_preprocessor.normalizedRelativePos('I Walk Away').mean()
# mean_2 = data_preprocessor.normalizedRelativePos('Walk Away').mean()
# print(mean_1)
# print(mean_2)


AttributeError: 'NoneType' object has no attribute 'mean'

### Speaker and "Walk Away/I Walk Away" Correlation"

In [ ]:
data_preprocessor.gettMatchedUtterancesDF("Walk Away").head()

AttributeError: 'DataPreprocesser' object has no attribute 'gettMatchedUtterancesDF'

### Accept Deal vs. Reject Deal

In [ ]:
# Using the filterRows method from DataPreprocessor
filtered_rows_submitted = data_preprocessor.filterRows("formattedChat", "Submitted agreement", case_in=False)
print("Submitted agreement:", filtered_rows_submitted.shape)

filtered_rows_reject = data_preprocessor.filterRows("formattedChat", "Reject Deal", case_in=False)
print("reject:", filtered_rows_reject.shape)

filtered_rows_reject_U = data_preprocessor.filterRows("formattedChat", "Reject Deal", case_in=True)
print("Reject (case-sensitive):", filtered_rows_reject_U.shape)

filtered_rows_accept = data_preprocessor.filterRows("formattedChat", "Accept Deal", case_in=False)
print("accept:", filtered_rows_accept.shape)

filtered_rows_accept_U = data_preprocessor.filterRows("formattedChat", "Accept Deal", case_in=True)
print("Accept (case-sensitive):", filtered_rows_accept_U.shape)

# Count exact matches between case-sensitive and case-insensitive searches
exact_matches = (filtered_rows_accept["formattedChat"] == filtered_rows_accept_U["formattedChat"]).sum()
print("Exact case matches:", exact_matches)

# Calculate total valid responses
total_valid_responses = filtered_rows_accept.shape[0] + filtered_rows_reject.shape[0]
print("Total Valid Responses:", total_valid_responses)

# Check for intersection between accept and reject
accept_reject_intersection = filtered_rows_accept.index.intersection(filtered_rows_reject.index)
print("Accept-Reject Intersection:", accept_reject_intersection.shape)


# filtered_rows_accept["formattedChat"].iloc[1]
# filtered_rows_submitted["formattedChat"].iloc[0]

### Potentially Flawed "Submitted Agreement" outcomes from conversation logic

Some of the formatted chats have a submitted agreement that does not match the context of the conversation.

Ex: This dialogue memtions the buyer recieved a 30% refund, but the submitted agreement states "Buyer gets no refund".
   
     "nan Seller: Your sudden demand for a refund is unwarranted. Our product description is crystal clear, and we stand by our policy. Your behavior is disappointing, and your negative review is unfounded.
    1699391248 Buyer: your sudden change and basic lying of the product is unwarranted. i am asking for a refund due to what i was expecting wasnt at all what was issued in the ad. so the crystal clear buyer seller negotiated should be void
    1699391270 Seller: Our product was as described on the website. However, if you remove your negative comment, I can reconsider a 30% refund. Let this be a resolution.
    1699391372 Buyer: i will accept that deal as long as the negative review of myslef and comment are removed
    1699391395 Seller: I agree with that resolution. Once your negative review is removed, I'll withdraw my negative review of you and process a 30% refund. Let's move forward from this.
    1699391413 Buyer: agreed
    1699391433 Seller: I appreciate your decision. I'll initiate the refund process and remove the negative review promptly. Let's keep interactions respectful and professional in future.
    1699391478 Buyer: absolutely and hopefully we can do honest and professional business in the future  good day.
    1699391494 Seller: Certainly, maintaining a professional relationship is pivotal. Your refund will be processed soon. Have a good day too.
    1699391631 Buyer: yes i absolutley agree. i do a lot of online shopping
    1699391649 Seller: Great to hear! I'm confident future transactions will be smoother. Let's leave this behind and look forward to better experiences.
    1699391731 Buyer: an i am an avid yelp reviewer and blogger so my comments make it and have an affect on a lot of online shoppers and buyers
    1699391750 Seller: Fantastic! Your influence can certainly add to the reputation of online businesses. Let's consider this a learning experience and move forward positively.
    1699391752 Buyer: Submitted agreement: Buyer gets no refund, buyer retracted their review, seller kept their review, buyer did apologize, and seller didn't apologize."

Ex: This one contains a dialogue that starts with Seller, and has the prompt for Gemini

    nan Seller: Your sudden demand for a refund is unwarranted. Our product description is crystal clear, and we stand by our policy. Your behavior is disappointing, and your negative review is unfounded.
    1718876388 Buyer: The description was misleading. I have proof it advertised a Kobe Bryant jersey. I just want the jersey I originally purchased for my nephew.
    1718876416 Seller: Our listing did not mention Kobe Bryant at all, any misunderstanding is on you. I can give you a partial refund, but only if you remove your false review.
    1718876489 Buyer:  See the latest updates to the Gemini Apps Privacy HubOpens in a new window
    close
    Conversation with Gemini

    Describe the flavor of a tomato

    volume_up
    A tomato's flavor is a complex dance between sweetness and acidity. Here's a breakdown:

    Sweetness: Comes from natural sugars like fructose and glucose.
    Acidity: Stems from citric and malic acids, giving tomatoes a tangy or tart kick.
    These elements combine to create a flavor profile that can be described as:

    Sweet-tart: This is the classic tomato flavor, where the sweetness and acidity are balanced.
    Sweeter: Some tomato varieties, especially cherry tomatoes, are bred to have a higher sugar content, making them taste sweeter.
    More acidic: These tomatoes have a stronger tang or sourness.
    Bland: If a tomato has low levels of both sugar and acid, it might taste bland.
    There are other factors that influence tomato flavor:

    Variety: There are hundreds of tomato varieties, each with its own unique flavor profile. For instance, some heirloom tomatoes are known for their intense, complex flavors.
    Ripeness: A perfectly ripe tomato will have the best flavor balance. Underripe tomatoes can be tart, while overripe ones may lose their acidity.
    Overall, the flavor of a tomato is a delightful combination of sweet and tart, with variations depending on the variety and ripeness.




    share


    more_vert
    expand_more
    Role Play Instructions:  You will play the role of a mistreated buyer in a purchase dispute with another player online. Imagine you are in the following situation:

    Kobe Bryant Jersey (You are the Buyer)

    Your terminally ill nephew is a huge Kobe Bryant fan so you purchased him a replica of Bryant’s last Basketball Championship jersey for $75. The website clearly indicated the purchase was for Bryant’s jersey. Other sites were available but this was cheaper and offered quick delivery. When the jersey arrived, it was for a different player you never heard of. You request the correct jersey be sent.

    The Seller responds: “The website clearly indicated this was for a Los Angeles Lakers jersey, not for a specific player. All sales are final.” You see they now removed mention of Kobe Bryant from their website but you know they are lying. To protect other customers, you post a negative review warning about the Seller’s deceptive behavior.

    The Seller posted a negative review about you, calling you a “SMARTASS, SLANDERER and a FRAUD.” You have dozens of transactions on this site and have a near-spotless reputation. Now you worry others won’t sell to you.

    volume_up
    You (the Buyer): Upset but Determined
    Your Demeanor: You're angry and frustrated, but you're also determined to get this resolved for your nephew and warn others.

    Your Goals:

    Get the Correct Jersey: You want the seller to send the Kobe Bryant jersey you originally purchased.
    Clear Your Name: You want the seller's negative review removed and your reputation restored.
    Warn Others: You want to make sure other potential buyers are aware of the seller's deceptive practices.
    What You Know:

    You have documented proof of your purchase, clearly showing it advertised a Kobe Bryant jersey.
    You have screenshots of the website before they removed the Kobe Bryant reference.
    You have a positive reputation on the website with many successful transactions.
    What You Can Do:

    Contact the Seller Again: State your case calmly but firmly. Remind them of the misleading advertisement and request the correct jersey be sent.
    Report the Seller: Report the seller's deceptive practices and false advertising to the website's customer service department. Include your purchase proof and screenshots.
    Respond to the Seller's Review: Write a public response to the seller's review on your profile. State the facts of the situation, provide evidence (if allowed by the website), and emphasize your positive purchase history.
    Dispute the Review: If possible, file a dispute against the seller's review. Explain the situation and provide evidence to support your claim.
    Here are some phrases you can use:

    "I am very disappointed with this purchase. The website clearly advertised a Kobe Bryant jersey, and that's what I paid for."
    "I am requesting that you send me the correct jersey as advertised."
    "I have documented proof that the website advertised a Kobe Bryant jersey at the time of purchase."
    "Your response and false advertising are unacceptable. I have reported your behavior to the website's customer service department."
    "The negative review you left on my profile is completely false and defamatory. My purchase history clearly shows my trustworthiness."
    "I am a loyal customer with a positive reputation on this website. This situation does not reflect my usual experience."
    Remember:

    Stay calm and professional, even though you're upset.
    Focus on the facts and provide evidence to support your claims.
    Be persistent in getting the issue resolved and your reputation restored.
    By following these steps, you can hopefully get the Kobe Bryant jersey for your nephew and clear your name on the website.




    share


    more_vert
    expand_more
    Issues to discuss
    Below are issues you could negotiate to resolve this dispute. Think about reasons why these might be important to you. For each issue, write a sentence justifying why you deserve this.  Be speciffic!! You can later use these reasons when you negotiate.

    volume_up
    Issues for Negotiation:
    1. Receiving the Correct Jersey:

    Reasoning: This is the most important issue. My terminally ill nephew is a huge Kobe Bryant fan, and I specifically purchased the jersey for him based on the advertised description. Not receiving the correct jersey defeats the entire purpose of the purchase and causes emotional distress for my nephew during a difficult time.
    2. Expedited Shipping:

    Reasoning: Considering the situation with my nephew, receiving the correct jersey as quickly as possible is crucial. Since the seller is at fault for the mistake, they should be responsible for any expedited shipping costs to ensure my nephew can enjoy the jersey before it's too late.
    3. Partial Refund:

    Reasoning: The website advertised a specific product (Kobe Bryant jersey) at a specific price. Due to the seller's deceptive practices, I was not given the product I paid for. A partial refund would acknowledge the inconvenience and wasted time caused by this situation.
    4. Seller Review Removal:

    Reasoning: The seller's negative review ("SMARTASS, SLANDERER and a FRAUD") is completely false and tarnishes my reputation on the website. Its removal is essential to restore my good standing and ensure other sellers are willing to do business with me.
    5. Public Apology from Seller:

    Reasoning: The seller's deceptive advertising and false review not only hurt me but potentially misled other customers. A public apology would acknowledge their wrongdoing and deter them from similar behavior in the future.
    Remember: While all these issues are important, prioritize getting the Kobe Bryant jersey for your nephew as quickly as possible.




    share


    more_vert

    Receive a refund: You could demand a full or partial refund for the price of the item. Why might this be important to you? (1 sentence)

    volume_up
    A full refund wouldn't be necessary if I receive the Kobe jersey, but a partial refund would be important to compensate for the time wasted and potential additional costs (expedited shipping) caused by the seller's mistake.




    share


    more_vert

    Keep your negative review of Seller: The Seller might demand you take down your negative review. Why might you want to keep it up on the site? (1 sentence)

    volume_up
    Keeping my negative review up is important to warn other potential buyers about the seller's deceptive advertising and false claims that could negatively impact their purchases.




    share


    more_vert

    Seller removes negative review of you: You could demand the Seller remove the bad review they made about you. Why might this be important to you? (1 sentence)

    volume_up
    Getting the seller's negative review removed is crucial because it's demonstrably false and harms my reputation on the website, potentially causing other sellers to hesitate to do business with me.




    share


    more_vert
    expand_more
    Receive formal apology: You could demand Seller update their bad review of you with an apology clarifying that they made a mistake and apologize for how they initially responded. Why might this be important to you? (1 sentence)

    volume_up
    A formal apology from the seller publicly acknowledges their wrongdoing, clears my name, and deters them from future deceptive practices that could harm other buyers.




    share


    more_vert
    expand_more
    On the next page, you will be matched with someone playing the other role in this dispute. This may take a few minutes. A sound will play when you are matched. Please ensure your volume is up.

    You will use the interface below. On the left is a chat window where you can send messages to the other side. You must take turns and can only send a message when it is your turn. You and your partner must exchange at least 8 messages. (Please respond quickly as your partner is waiting for you!)
    If you reach a tentative deal to resolve the dispute, use the menu on the right to clarify the terms of the agreement. If you can’t reach a deal, you can walk away from the negotiation by pressing “Walk Away”.

    volume_up
    I understand. I'm ready to chat with the seller when you connect me.




    share


    more_vert
    expand_more
    Give a simple response to this
    Other]: Your sudden demand for a refund is unwarranted. Our product description is crystal clear, and we stand by our policy. Your behavior is disappointing, and your negative review is unfounded.

    volume_up
    The description was misleading. I have proof it advertised a Kobe Bryant jersey. I just want the jersey I originally purchased for my nephew.




    share


    more_vert
    edit
    Other]: Our listing did not mention Kobe Bryant at all, any misunderstanding is on you. I can give you a partial refund, but only if you remove your false review

    Show drafts
    volume_up
    That's not accurate. I have screenshots showing the listing advertised a Kobe Bryant jersey before you changed it.  A partial refund wouldn't make my nephew happy,  but I'm open to discussing getting the right jersey sent quickly   and  the removal of your unfair review.
    1718876514 Seller: We both seem at a stalemate. This reflects poorly on both our reputations. Fine, I will offer you full refund if you withdraw your review.
    1718876649 Buyer: We both seem at a stalemate. This reflects poorly on both our reputations. Fine, I will offer you full refund if you withdraw your review
    1718876678 Seller: While I appreciate your offer, my main concern is getting the Kobe Bryant jersey for my nephew. If u can get the correct jersey shipped quickly, I'll remove the review. Deal?
    1718876767 Buyer: Absolutely, that sounds like a fair solution!  My nephew will be thrilled.  Please confirm you can expedite the Kobe Bryant jersey to me.  Once I receive confirmation and tracking information, I'll happily remove my review.




    tune

    share


    more_vert


    expand_content
    add_photo_alternatephoto_camera

    mic
    send
    Gemini may display inaccurate info, includin
    1718876790 Seller: Ok deal. Will send Kobe jersey with expedite shipping, and you'll remove your review. Let's both move on from this unpleasant situation.
    1718876872 Buyer: Submitted agreement: Buyer gets partial refund, buyer kept their review, seller kept their review, buyer did apologize, and seller didn't apologize.


# Testing Utterance Builder

# Testing Parser

In [ ]:
import re
text = data["formattedChat"][2]
print(text)
pattern = re.compile(r'^\s*(\d+|nan)?\s*(Buyer|Seller):\s*(.*)$', re.IGNORECASE)
structured_dialog = []

for line in str(text).split('\n'):
    line = line.strip()
    print(line)
    if not line:
        continue
    
    match = pattern.match(line)
 
    if match:
        timestamp_str, speaker, message = match.groups()
        print("Match found!")
        print("Timestamp:", timestamp)
        print("Speaker:", speaker)
        print("Message:", message)
    else:
        print("Match NOT found!")
        print(match)
        # print("Timestamp:", timestamp)
        # print("Speaker:", speaker)
        # print("Message:", message)